In [170]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [171]:
import sys
import os
import pandas as pd
import numpy as np
import math
import datetime as dt

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

from itertools import cycle
import plotly.graph_objects as go # add details
import plotly.express as px
from plotly.subplots import make_subplots

import seaborn as sns 
import matplotlib.pyplot as plt 
from colorama import Fore #for adding colours

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

# Add the project root directory to the system path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
# Import the Transformer model from transformer.py
from models.transformer import get_model

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

ImportError: cannot import name 'get_model' from 'models.transformer' (d:\khtn\Cac_cong_nghe_moi_trong_phat_trien_phan_mem\Do_an\Ca_nhan\crypto-prediction\models\transformer.py)

In [172]:
# Example data loading
df = pd.read_csv(r"../data/BTC-USD.csv")  # Replace with your actual data path

# Convert the date column to datetime if not already
df['date'] = pd.to_datetime(df['date'])

# Extract useful date features
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['day_of_week'] = df['date'].dt.dayofweek

# Separate the date features
time_features = df[['year', 'month', 'day', 'day_of_week']].values

# Drop the original date column and time features
df = df.drop(columns=['date', 'year', 'month', 'day', 'day_of_week'])

# Add a dummy implementation of technical indicators (Replace with actual calculation)
df['roc'] = np.random.random(len(df))  # Dummy ROC
df['rsi'] = np.random.random(len(df))  # Dummy RSI
df['bollinger_bands'] = np.random.random(len(df))  # Dummy Bollinger Bands
df['moving_average'] = np.random.random(len(df))  # Dummy Moving Average
df['support_resistance'] = np.random.random(len(df))  # Dummy Support/Resistance

# Ensure selected features are in the DataFrame
features = [feature for feature in selected_features if feature in df.columns]

# Select only the required features
data = df[features].values

# Ensure the target (closing price) is always included
targets = df['closing_price'].values

# Ensure the data is of the correct type
data = data.astype(np.float32)
targets = targets.astype(np.float32)
time_features = time_features.astype(np.float32)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, targets, test_size=0.2, random_state=42)
time_train, time_test = train_test_split(time_features, test_size=0.2, random_state=42)

# Create TensorDatasets
train_dataset = TensorDataset(torch.tensor(X_train), torch.tensor(time_train), torch.tensor(y_train))
test_dataset = TensorDataset(torch.tensor(X_test), torch.tensor(time_test), torch.tensor(y_test))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define and initialize the Transformer model
from models.transformer import get_model

# Adjust input_dim based on the number of selected features
input_dim = len(features)
time_dim = 4  # Number of time features, assuming year, month, day, and day_of_week

model = TimeSeriesTransformer(input_dim, time_dim, 64, 8, 4, 1)

# Training loop
def train_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        for batch in train_loader:
            inputs, time_inputs, targets = batch
            optimizer.zero_grad()
            outputs = model(inputs, time_inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 50

train_model(model, train_loader, criterion, optimizer, num_epochs)

# Make predictions
model.eval()
predictions = []
with torch.no_grad():
    for batch in test_loader:
        inputs, time_inputs, _ = batch
        outputs = model(inputs, time_inputs)
        predictions.append(outputs.numpy())

# Flatten the predictions list
predictions = np.concatenate(predictions)

# Plot predictions
plt.figure(figsize=(10,5))
plt.plot(y_test, label='True Prices')
plt.plot(predictions, label='Predicted Prices')
plt.legend()
plt.show()


KeyError: 'date'